<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#하이퍼파라미터-세팅-및-seed-고정" data-toc-modified-id="하이퍼파라미터-세팅-및-seed-고정-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>하이퍼파라미터 세팅 및 seed 고정</a></span></li><li><span><a href="#학습-데이터-EDA" data-toc-modified-id="학습-데이터-EDA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>학습 데이터 EDA</a></span></li><li><span><a href="#데이터-전처리-함수-정의-(Dataset)" data-toc-modified-id="데이터-전처리-함수-정의-(Dataset)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>데이터 전처리 함수 정의 (Dataset)</a></span></li><li><span><a href="#Dataset-정의-및-DataLoader-할당" data-toc-modified-id="Dataset-정의-및-DataLoader-할당-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dataset 정의 및 DataLoader 할당</a></span><ul class="toc-item"><li><span><a href="#데이터-샘플-시각화-(Show-example-image-and-mask)" data-toc-modified-id="데이터-샘플-시각화-(Show-example-image-and-mask)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>데이터 샘플 시각화 (Show example image and mask)</a></span></li></ul></li><li><span><a href="#baseline-model" data-toc-modified-id="baseline-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>baseline model</a></span><ul class="toc-item"><li><span><a href="#[TODO]-코드-구현-FCN-8s-" data-toc-modified-id="[TODO]-코드-구현-FCN-8s--5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span><font color="red">[TODO] 코드 구현 FCN-8s </font></a></span></li></ul></li><li><span><a href="#train,-validation,-test-함수-정의" data-toc-modified-id="train,-validation,-test-함수-정의-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>train, validation, test 함수 정의</a></span></li><li><span><a href="#모델-저장-함수-정의" data-toc-modified-id="모델-저장-함수-정의-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>모델 저장 함수 정의</a></span></li><li><span><a href="#모델-생성-및-Loss-function,-Optimizer-정의" data-toc-modified-id="모델-생성-및-Loss-function,-Optimizer-정의-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>모델 생성 및 Loss function, Optimizer 정의</a></span></li><li><span><a href="#저장된-model-불러오기-(학습된-이후)" data-toc-modified-id="저장된-model-불러오기-(학습된-이후)-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>저장된 model 불러오기 (학습된 이후)</a></span></li><li><span><a href="#submission을-위한-test-함수-정의" data-toc-modified-id="submission을-위한-test-함수-정의-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>submission을 위한 test 함수 정의</a></span></li><li><span><a href="#submission.csv-생성" data-toc-modified-id="submission.csv-생성-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>submission.csv 생성</a></span></li><li><span><a href="#Reference" data-toc-modified-id="Reference-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

In [7]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from utils import label_accuracy_score
import cv2

import numpy as np
import pandas as pd

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.8.0.dev20210130
GPU 사용 가능 여부: True
GeForce GTX 1080 Ti
2


## 하이퍼파라미터 세팅 및 seed 고정

In [2]:
batch_size = 8   # Mini-batch size
num_epochs = 20
learning_rate = 0.0001

In [3]:
# seed 고정
random_seed = 21
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## baseline model

### <font color='red'>[TODO] 코드 구현 FCN-8s </font>

In [5]:
# 모델 참고 코드 
# https://github.com/wkentaro/pytorch-fcn/
import torch
import torch.nn as nn

class FCN8s(nn.Module):
    def __init__(self, num_classes=12):
        super(FCN8s, self).__init__()
        self.num_classes = num_classes
        # convolution block (3x3 and 1x1)
        def conv_block(in_channel, out_channel, k=3, s=1, p=1, pool=True):
            if pool:
                return nn.Sequential(nn.Conv2d(in_channel, out_channel, k, s, p), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2, ceil_mode=True))
            else:
                return nn.Sequential(nn.Conv2d(in_channel, out_channel, k, s, p), nn.ReLU(inplace=True))
        
        def conv_1x1(in_channel, out_channel):
            return nn.Sequential(nn.Conv2d(in_channel, out_channel, 1, 1, 0), nn.ReLU(inplace=True))
        

        # stack convolution blocks (1~7)
        channels = [3, 64, 128, 256, 512, 512, 4096, 4096]
        conv_layers = []
        conv_layer_names = []
        for i in range(len(channels)-1):
            # conv block 1~5 (3x3 conv)
            if i < 2: # block 1~2
                conv_layer_names += [f'conv{i+1}_1', f'conv{i+1}_2']
                conv_layers.append(conv_block(channels[i], channels[i+1], pool=False))
                conv_layers.append(conv_block(channels[i+1], channels[i+1]))
            elif 1 < i < 5: # block 3~5
                conv_layer_names += [f'conv{i+1}_1', f'conv{i+1}_2', f'conv{i+1}_3']
                conv_layers.append(conv_block(channels[i], channels[i+1], pool=False))
                conv_layers.append(conv_block(channels[i+1], channels[i+1], pool=False))
                conv_layers.append(conv_block(channels[i+1], channels[i+1]))
            # FC block 6~7 (1x1 conv)
            else:
                conv_layer_names += [f'FC{i+1}']
                conv_layers.append(conv_block(channels[i], channels[i+1], 1, 1, 0, pool=False))
        
        # score layer (1x1 conv)
        conv_layer_names += [f'score']
        conv_layers.append(conv_1x1(channels[i+1], self.num_classes))
        
        self.net = nn.Sequential()
        for layer, name in zip(conv_layers, conv_layer_names):
            self.net.add_module(name, layer)
                
        # upsampling
        self.upscore2 = nn.ConvTranspose2d(self.num_classes, self.num_classes, 4, 2, 1)
        self.upscore8 = nn.ConvTranspose2d(self.num_classes, self.num_classes, 16, 8, 4)

        # branches
        sequential_layers = list(self.net.children())
        self.branch1 = nn.Sequential(*sequential_layers[:10])
        self.branch2 = nn.Sequential(*sequential_layers[:7])
        
        # summation
        self.conv1x1_1 = conv_1x1(512, self.num_classes)
        self.conv1x1_2 = conv_1x1(256, self.num_classes)

    def forward(self, x):
        def conv_1x1(in_channel, out_channel):
            return nn.Sequential(nn.Conv2d(in_channel, out_channel, 1, 1, 0), nn.ReLU(inplace=True))

        sequential_output = self.net(x)
        output_1 = self.conv1x1_1(self.branch1(x)) + self.upscore2(sequential_output)
        output_2 = self.conv1x1_2(self.branch2(x)) + self.upscore2(output_1)
        output = self.upscore8(output_2)

        return output

        

In [68]:
model = FCN8s(num_classes=12)
print(model)

FCN8s(
  (net): Sequential(
    (conv1_1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
    )
    (conv1_2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (conv2_1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
    )
    (conv2_2): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (conv3_1): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
    )
    (conv3_2): Sequential(
      (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=

In [6]:
# 구현된 model에 임의의 input을 넣어 output이 잘 나오는지 test

model = FCN8s(num_classes=12)
x = torch.randn([1, 3, 512, 512])
print("input shape : ", x.shape)
out = model(x).to(device)
print("output shape : ", out.size())

model = model.to(device)

input shape :  torch.Size([1, 3, 512, 512])
output shape :  torch.Size([1, 12, 512, 512])
